In [1]:
# Import necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pytorch_lightning as pl
import torch
from torch.utils.data import Dataset, DataLoader
from multiprocessing import cpu_count
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import torch.autograd as autograd
import torchmetrics
import torch.optim as optim
import matplotlib.pyplot as plt   # plotting
import seaborn as sns   # plotting heatmap
from src.classification import utils
from src import config
import os

from src.utils import round_down_to_closest_even, convert_dict_to_series
from src.statistical_analysis.business.DistanceFocusingLogic import recenter_distances, cut_tail, cut_prefix

%matplotlib inline

In [2]:
def get_all_valid_subject_data_df():
    df_nap = pd.read_pickle(os.path.join(os.path.join(config.get_project_root(), "resources", "nap", "data"), config.RAW_GAZE_FILE))
    df_no_nap = pd.read_pickle(os.path.join(os.path.join(config.get_project_root(), "resources", "no_nap", "data"), config.RAW_GAZE_FILE))
    df = df_nap.append(df_no_nap)
    valid_df = df[df.notnull().all(1)]
    return valid_df

In [3]:
df = get_all_valid_subject_data_df()
rois = utils.get_aggregated_roi_df()
df

C:\Users\Daniel\AppData\Local\Temp\11\ipykernel_39196\3133104631.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_nap.append(df_no_nap)


Pupil radius Measured Eye  \
Subject Session   Movie Memory TimeStamp                              
AL5     Session A mov1  -4     0                7082.0            R   
                               2                7079.0            R   
                               4                7078.0            R   
                               6                7079.0            R   
                               8                7085.0            R   
...                                                ...          ...   
YP2     Session B mov9  -2     6722             4409.0            R   
                               6724             4403.0            R   
                               6726             4396.0            R   
                               6728             4397.0            R   
                               6730             4399.0            R   

                                              X_gaze      Y_gaze  is Blink  \
Subject Session   Movie Memory TimeStamp                                     
AL5     Session A mov1  -4     0          821.530208  261.511111     False   
                               2          821.441250  261.911111     False   
                               4          821.886042  262.533333     False   
                               6          821.930521  262.400000     False   
                               8          821.930521  262.533333     False   
...                                              ...         ...       ...   
YP2     Session B mov9  -2     6722       323.674896   98.933333     False   
                               6724       323.808333   98.666667     False   
                               6726       323.808333   98.355556     False   
                               6728       323.052187   98.133333     False   
                               6730       322.874271   98.044444     False   

                                          is Fixation    Distance        DVA  
Subject Session   Movie Memory TimeStamp                                      
AL5     Session A mov1  -4     0                False  443.627326  11.016477  
                               2                False  443.241295  11.006747  
                               4                False  442.951696  10.998814  
                               6                False  443.088202  11.002239  
                               8                False  442.975424  10.999364  
...                                               ...         ...        ...  
YP2     Session B mov9  -2     6722             False  301.043420   7.491555  
                               6724             False  301.211334   7.495977  
                               6726             False  301.483154   7.502890  
                               6728             False  302.045508   7.516340  
                               6730             False  302.209917   7.520321  

[42938882 rows x 8 columns]

In [4]:
roi_drop_movies = set(rois.index) - set(config.valid_movies)
distance_drop_movies = roi_drop_movies.union({
    f'mov{idx}' for idx in range(config.num_repeating_movies + 1, config.total_recorded_movies + 1)})
valid_rois = rois.drop(roi_drop_movies)
valid_df = df.drop(index=distance_drop_movies, level=config.MOVIE, errors='ignore')

In [5]:
valid_df.head()

Pupil radius Measured Eye  \
Subject Session   Movie Memory TimeStamp                              
AL5     Session A mov1  -4     0                7082.0            R   
                               2                7079.0            R   
                               4                7078.0            R   
                               6                7079.0            R   
                               8                7085.0            R   

                                              X_gaze      Y_gaze  is Blink  \
Subject Session   Movie Memory TimeStamp                                     
AL5     Session A mov1  -4     0          821.530208  261.511111     False   
                               2          821.441250  261.911111     False   
                               4          821.886042  262.533333     False   
                               6          821.930521  262.400000     False   
                               8          821.930521  262.533333     False   

                                          is Fixation    Distance        DVA  
Subject Session   Movie Memory TimeStamp                                      
AL5     Session A mov1  -4     0                False  443.627326  11.016477  
                               2                False  443.241295  11.006747  
                               4                False  442.951696  10.998814  
                               6                False  443.088202  11.002239  
                               8                False  442.975424  10.999364

In [6]:
df = cut_tail(valid_df, valid_rois)

In [7]:
def cut_prefix(distances, rois, starting_point, verbose=False):
    distances_movies = set(distances.index.unique(level=config.MOVIE))
    rois_movies = set(rois.index.unique())
    assert (distances_movies == rois_movies), f'Movies in Distances Series do not match the Movies in RoIs data.\n\tDistances Movies: {distances_movies}\n\tRoIs Movies: {rois_movies}'

    event_times = rois['t_median']
    movies = distances.index.unique(level=config.MOVIE)
    for movID in movies:
        assert (movID in distances.index.unique(level=config.MOVIE)), f'Couldn\'t find distances for movie {movID}'
        assert (movID in rois.index.unique()), f'Couldn\'t find RoI data for movie {movID}'
        event_time = round_down_to_closest_even(event_times[movID])
        time_cond = distances.index.get_level_values(config.TIMESTAMP) > event_time - starting_point
        movie_cond = distances.index.get_level_values(config.MOVIE) != movID
        distances = distances.loc[movie_cond | time_cond]
        if verbose:
            print(f'Finished cutting distance prefix for Movie {movID}.')
    return distances

In [8]:
df = cut_prefix(df, valid_rois, 1500)

In [9]:
relevant_couples = set(zip(df.index.get_level_values(config.SUBJECT),
                           df.index.get_level_values(config.MOVIE)))
series_id_df = pd.DataFrame(relevant_couples, columns=[config.SUBJECT, config.MOVIE])
series_id_df['series_id'] = series_id_df.index
series_id_df

,Subject,Movie,series_id
0,AI5,mov61,0
1,SC2,mov80,1
2,AH3,mov65,2
3,MZ2,mov75,3
4,NM9,mov80,4
...,...,...,...
2587,AM2,mov11,2587
2588,KS0,mov2,2588
2589,EF4,mov13,2589
2590,YN5,mov21,2590


In [10]:
df = df.reset_index()
df

,Subject,Session,Movie,Memory,TimeStamp,Pupil radius,Measured Eye,X_gaze,Y_gaze,is Blink,is Fixation,Distance,DVA
0,AL5,Session A,mov1,-4,4438,5368.0,R,441.544688,304.977778,False,True,362.109842,9.044441
1,AL5,Session A,mov1,-4,4440,5366.0,R,442.434271,305.200000,False,True,361.552798,9.031140
2,AL5,Session A,mov1,-4,4442,5362.0,R,442.211875,304.933333,False,True,361.885745,9.039374
3,AL5,Session A,mov1,-4,4444,5366.0,R,442.345312,305.066667,False,True,361.710460,9.035054
4,AL5,Session A,mov1,-4,4446,5365.0,R,442.701146,305.244444,False,True,361.406383,9.027674
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3718246,YP2,Session B,mov80,-3,5576,3691.0,R,849.666667,579.133333,False,True,460.603454,11.205830
3718247,YP2,Session B,mov80,-3,5578,3688.0,R,850.400000,579.466667,False,True,459.805761,11.186544
3718248,YP2,Session B,mov80,-3,5580,3686.0,R,851.066667,579.733333,False,True,459.100129,11.169519
3718249,YP2,Session B,mov80,-3,5582,3685.0,R,851.133333,578.866667,False,True,459.509792,11.180311


In [11]:
new_df = df.merge(series_id_df, on = [config.SUBJECT, config.MOVIE])
new_df

,Subject,Session,Movie,Memory,TimeStamp,Pupil radius,Measured Eye,X_gaze,Y_gaze,is Blink,is Fixation,Distance,DVA,series_id
0,AL5,Session A,mov1,-4,4438,5368.0,R,441.544688,304.977778,False,True,362.109842,9.044441,1980
1,AL5,Session A,mov1,-4,4440,5366.0,R,442.434271,305.200000,False,True,361.552798,9.031140,1980
2,AL5,Session A,mov1,-4,4442,5362.0,R,442.211875,304.933333,False,True,361.885745,9.039374,1980
3,AL5,Session A,mov1,-4,4444,5366.0,R,442.345312,305.066667,False,True,361.710460,9.035054,1980
4,AL5,Session A,mov1,-4,4446,5365.0,R,442.701146,305.244444,False,True,361.406383,9.027674,1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3718246,YP2,Session B,mov80,-3,5576,3691.0,R,849.666667,579.133333,False,True,460.603454,11.205830,111
3718247,YP2,Session B,mov80,-3,5578,3688.0,R,850.400000,579.466667,False,True,459.805761,11.186544,111
3718248,YP2,Session B,mov80,-3,5580,3686.0,R,851.066667,579.733333,False,True,459.100129,11.169519,111
3718249,YP2,Session B,mov80,-3,5582,3685.0,R,851.133333,578.866667,False,True,459.509792,11.180311,111


In [12]:
df = new_df[['series_id', config.gaze_X, config.gaze_Y, config.PUPIL, config.DVA, config.SESSION]]
df[config.SESSION] = (df[config.SESSION] == config.SESSION_B).astype(int)
df

C:\Users\Daniel\AppData\Local\Temp\11\ipykernel_39196\3789580647.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[config.SESSION] = (df[config.SESSION] == config.SESSION_B).astype(int)


,series_id,X_gaze,Y_gaze,Pupil radius,DVA,Session
0,1980,441.544688,304.977778,5368.0,9.044441,0
1,1980,442.434271,305.200000,5366.0,9.031140,0
2,1980,442.211875,304.933333,5362.0,9.039374,0
3,1980,442.345312,305.066667,5366.0,9.035054,0
4,1980,442.701146,305.244444,5365.0,9.027674,0
...,...,...,...,...,...,...
3718246,111,849.666667,579.133333,3691.0,11.205830,1
3718247,111,850.400000,579.466667,3688.0,11.186544,1
3718248,111,851.066667,579.733333,3686.0,11.169519,1
3718249,111,851.133333,578.866667,3685.0,11.180311,1


In [13]:
# df_dummies = pd.get_dummies(new_df, columns=[config.SUBJECT, config.MOVIE], drop_first=True)
# df_dummies.sample(10).style.background_gradient(cmap = 'Blues')

In [14]:
FEATURE_NAMES = [config.gaze_X, config.gaze_Y, config.PUPIL, config.DVA]

In [15]:
sequences = []
for series_id, group in df.groupby("series_id"):
    sequence_feature = group[FEATURE_NAMES]
    label = df[df.series_id == series_id].iloc[0][config.SESSION]
    sequences.append((sequence_feature, label))

sequences[0]

(             X_gaze      Y_gaze  Pupil radius       DVA
 1728172  947.600000  392.733333        5585.0  8.086982
 1728173  947.333333  393.333333        5582.0  8.083205
 1728174  946.400000  394.000000        5580.0  8.091396
 1728175  947.066667  394.066667        5585.0  8.077483
 1728176  947.200000  394.600000        5587.0  8.066999
 ...             ...         ...           ...       ...
 1729589  970.666667  435.400000        5419.0  7.025778
 1729590  971.400000  435.466667        5419.0  7.009971
 1729591  971.266667  434.733333        5421.0  7.022479
 1729592  970.733333  434.466667        5423.0  7.036880
 1729593  970.066667  435.133333        5419.0  7.041539
 
 [1422 rows x 4 columns],
 0.0)

In [16]:
train_seq, test_seq = train_test_split(sequences, random_state=420, test_size=0.2)
len(train_seq), len(test_seq)

(2073, 519)

Dataset

In [17]:
class EyeTrackingDataset(Dataset):

    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence, label = self.sequences[idx]
        return dict(
            sequences=torch.Tensor(sequence.to_numpy()),
            label=torch.tensor(label).long()
        )

In [18]:
class EyeTrackingDataModule(pl.LightningDataModule):

    def __init__(self, train_sequence, test_sequence, batch_size):
        super().__init__()
        self.train_sequence = train_sequence
        self.test_sequence = test_sequence
        self.batch_size = batch_size

    def setup(self, stage=None):
        self.train_sequence = EyeTrackingDataset(self.train_sequence)
        self.test_sequence = EyeTrackingDataset(self.test_sequence)

    def train_dataloader(self):
        return DataLoader(self.train_sequence, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.test_sequence, batch_size=self.batch_size,shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_sequence, batch_size=self.batch_size,shuffle=False)

In [19]:
N_EPOCHS = 250
BATCH_SIZE = 64 # ?

data_module = EyeTrackingDataModule(train_seq, test_seq, BATCH_SIZE)

Model

In [20]:
class SequenceModel(nn.Module):

    def __init__(self, n_features, n_classes, n_hidden=256, n_layers=3):
        super().__init__()

        self.n_hidden = n_hidden

        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=n_hidden,
            num_layers=n_layers,
            batch_first=True,
            dropout=0.75
        )

        self.classifier = nn.Linear(n_hidden, n_classes)

    def forward(self, x):
        self.lstm.flatten_parameters()
        _, (hidden, _) = self.lstm(x)

        out = hidden[-1]
        return self.classifier(out)

In [21]:
class EyeTrackingPredictor(pl.LightningModule):

    def __init__(self, n_features:int, n_classes:int):
        super().__init__()
        self.model = SequenceModel(n_features, n_classes)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x, labels=None):
        output = self.model(x)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels)
        return loss, output

    def training_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = torchmetrics.functional.accuracy(predictions, labels)

        self.log("train_loss", loss, prog_bar=True, logger=True)
        self.log("train_accuracy", step_accuracy, prog_bar=True, logger=True)
        return {"loss": loss, "accuracy": step_accuracy}

    def validation_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = torchmetrics.functional.accuracy(predictions, labels)

        self.log("validation_loss", loss, prog_bar=True, logger=True)
        self.log("validation_accuracy", step_accuracy, prog_bar=True, logger=True)
        return {"loss": loss, "accuracy": step_accuracy}

    def test_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = torchmetrics.functional.accuracy(predictions, labels)

        self.log("test_loss", loss, prog_bar=True, logger=True)
        self.log("test_accuracy", step_accuracy, prog_bar=True, logger=True)
        return {"loss": loss, "accuracy": step_accuracy}

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr = 0.0001)

In [22]:
model = EyeTrackingPredictor(
    n_features=len(FEATURE_NAMES),
    n_classes=2
)

In [23]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

Launching TensorBoard...

In [24]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

logger = TensorBoardLogger("lightning_logs", name="EyeTracking")

trainer = pl.Trainer(logger=logger, callbacks=checkpoint_callback, max_epochs=N_EPOCHS, enable_progress_bar=True)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [25]:
trainer.fit(model, data_module)

Missing logger folder: lightning_logs\EyeTracking

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | SequenceModel    | 1.3 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.286     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\Daniel\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


RuntimeError: stack expects each tensor to be equal size, but got [1498, 4] at entry 0 and [1443, 4] at entry 2